In [ ]:
# selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# other lib
import time
import os
import re
import numpy as np

# bs 
from bs4 import BeautifulSoup as bs
import requests

#data
import pandas as pd 



def item_extract(item_cont, key):
    parse = item_cont.select(key)
    if parse == []:
        return np.nan
    else:
        return parse[0].text



#get keys 
address_key = "a.business-name>span"
category_key = "div.categories:nth-child(2)>a:nth-child(2)"
cat_url_key = "div.categories:nth-child(2)>a:nth-child(2)"
order_key = "div.listing-ctas>a.action.order-online"
open_close_key = "div.info-section.info-secondary>div.open-status"
phone_key = "div.info-section.info-secondary>div.phones.phone.primary"
location_key = "div.adr>div.locality"
price_range_key = "div.info-section.info-secondary>div.price-range"
review_numb_key = "a.ta-rating-wrapper>span.ta-count"
rating_key = "span.fs-rating-wrapper>span.count"
review_key = "div.snippet>p.body.with-avatar"
website_key = "div.links>a.track-visit-website"
menu_key = "div.links>a.menu"

#create lists
address_lst = []
category_lst = []
cat_url_lst = []
order_lst = []
open_close_lst = []
phone_lst = []
location_lst = []
prince_range_lst = []
review_numb_lst = []
rating_lst = []
review_lst = []
website_lst = []
menu_lst = []






url = "https://www.yellowpages.com/" 
# create a driver
driver = webdriver.Chrome("/Users/yasmina/Documents/Nod Bootcamp/Codealongs/Week 5 Files (x86)\chromedriver.exe")
# bring us to the webpage
driver.get(url)
soup = bs(driver.page_source, 'lxml')


key = "div.on-focus.search-container>label>input"
agree = driver.find_element(By.CSS_SELECTOR, key)   #find button
agree.click()  

#find the search result 
search_key = "div.on-focus.search-container.on-focus>label>input"
recommend = driver.find_element(By. CSS_SELECTOR, search_key)
element = WebDriverWait(driver, 10).until(lambda x: x.find_element(By.CSS_SELECTOR, search_key)) 
element.send_keys("coffee shops")


#click using the search button
button = "form>button"
get = driver.find_element(By. CSS_SELECTOR, button)
get.click()

next = [i.text for i in soup.select("div.pagination>ul>li>a.next.ajax-page")]
button_n = "div.pagination>ul>li>a.next.ajax-page"

time.sleep(2.5)
soup = bs(driver.page_source, 'lxml')
containers = soup.select("div.srp-listing.clickable-area.mdm")
for i in range(0,36):
    
    for i in containers:
            #name
            address = item_extract(i, address_key)
            address_lst.append(address)

            #cat
            category = item_extract(i, category_key)
            category_lst.append(category)

            #order
            order = item_extract(i, order_key)
            order_lst.append(order)

            #open_close
            open_close = item_extract(i, open_close_key)
            open_close_lst.append(open_close)

            #phone
            phone = item_extract(i, phone_key)
            phone_lst.append(phone)

            #location
            location = item_extract(i, location_key)
            location_lst.append(location)

            #price
            price_range = item_extract(i, price_range_key)
            prince_range_lst.append(price_range)

            #reviews
            review_numb_lst.append(item_extract(i, review_numb_key))
            review_numb_lst = [re.sub(r"[()]", "", str(i)) for i in review_numb_lst if i is not None]

            #rating 
            rating = item_extract(i, rating_key)
            rating_lst.append(rating)

            #comments
            review = item_extract(i, review_key)
            review_lst.append(review)
            time.sleep(2) 


    get = driver.find_element(By. CSS_SELECTOR, button_n)
    get.click()
    time.sleep(5)

time.sleep(2.5)
soup = bs(driver.page_source, 'lxml')


#create series 
series_address = pd.Series(address_lst, name = "address")
series_location = pd.Series(location_lst, name = "location")
series_category = pd.Series(category_lst, name = "category")
#series_cat_url = pd.Series(cat_url_lst, name = "cat_url")
series_open_close = pd.Series(open_close_lst, name = "open_close")
series_phone = pd.Series(phone_lst, name = "phone")
series_price_range = pd.Series(prince_range_lst, name = "price range")
series_reviews_number = pd.Series(review_numb_lst, name = "reviews")
series_rating = pd.Series(rating_lst, name = "rating")
series_review = pd.Series(review_lst, name = "review")

#create dataframe and safe it
df = pd.concat([series_address, series_location, series_category, series_open_close, series_phone, series_price_range, series_reviews_number, series_rating, series_review], axis=1)
df
df.to_csv('concatenated_data.csv', index=False)
